<div style="text-align: justify"> 


In [ ]:
#| echo: false
import geopandas as gpd
import pandas as pd
import locale
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import display, Markdown
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.precision', 4)
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
glebas = gpd.read_file('../glebas-federais.gpkg', layer='glebas-mais-amazonia')
area_estudo = gpd.read_file('../glebas-federais.gpkg', layer='area-estudo')
uf = gpd.read_file('../glebas-federais.gpkg', layer='uf-brasil')
sr_ponto = gpd.read_file('../glebas-federais.gpkg', layer='sr_incra')
uc=gpd.read_file('../glebas-federais.gpkg', layer='uc')
ti=gpd.read_file('../glebas-federais.gpkg', layer='ti-portarias')
ti = ti.set_crs(4674, allow_override=True)
pa = gpd.read_file('../glebas-federais.gpkg', layer='pa_brasil')
tq = gpd.read_file('../glebas-federais.gpkg', layer='tq_brasil')
sigef = gpd.read_file('../glebas-federais.gpkg', layer='sigef-particular')
floresta=gpd.read_file('../glebas-federais.gpkg', layer='florestas-publicas')
mun = gpd.read_file('../glebas-federais.gpkg', layer='municipios-area-estudo')

uf_analise = 'Acre'
display(Markdown(f'# Análise de Sobreposição individual das Glebas Federais do estado do {uf_analise}'))

# Definição de cores
cor_gleba = '#e9cf68'
cor_SIGEF = '#e0bd1f'
cor_PA = '#998115'
cor_TI = '#f2d4d3'
cor_TQ = '#d7c0dd'
cor_UC = '#99eb65'
cor_Floresta = '#a7f5e0'
cor_massa_agua = '#0751ff'
cor_rodovias = '#e10c00'
cor_gleba_flo = '#D9E3A9'
cor_gleba_uc = '#C1DC66'
cor_gleba_ti = '#EFD9A5'
cor_gleba_pa = '#C1A83E'
cor_gleba_tq = '#CDAFD4'
cor_gleba_sigef = '#E4C643'


:::{.table-center}


In [ ]:
#| echo: false


glebas = glebas[glebas['uf_nome']==uf_analise]
lista = glebas.gid.to_list()

for item in lista:
    gleba = glebas[glebas['gid']==item]
    display(Markdown(f'## Gleba Analisada: {gleba.nome_gleba}'))
    gleba = gleba.rename(columns={'nome_gleba':'Nome da Gleba','area_ha':'Área (ha)'})
    display(gleba[['Nome da Gleba','Área (ha)']].to_html(index=False))

    display(Markdown('### Abrangência Municipal'))
    fig, ax = plt.subplots(figsize=(13,6))
    
    municipio_gleba = mun.sjoin(gleba, how='inner')
    gleba.plot(ax=ax, facecolor=cor_gleba, alpha=0.8)
    municipio_gleba.plot(ax=ax, facecolor='none')
    # LEGENDAS
    patch_list = []
    patch_list.append(patches.Patch(facecolor=cor_gleba, alpha=0.6, edgecolor='none', label='Gleba Federal', linewidth=1))
    # Creates a legend with the list of patches above.
    ax.legend(handles=patch_list, fontsize=10, loc='lower left',
            bbox_to_anchor = (1,0.75),title='Legenda', title_fontsize=12)
    plt.xlabel('Longitude (°)')
    plt.ylabel('Latitude (°)')
    plt.grid()
    plt.show()
    municipio_gleba = municipio_gleba.rename(columns={'cd_uf':'Código da UF', 'nm_uf':'Estado', 'sigla':'UF', 'cd_mun':'Código do Município', 'nm_mun':'Nome do Município'})
    display(municipio_gleba[['Código da UF', 'Estado', 'UF', 'Código do Município', 'Nome do Município']].to_html(index=False))

    display(Markdown('### Floresta Pública'))
    floresta_gleba = floresta.sjoin(gleba, how='inner')
    if floresta_gleba.shape[0]>0:
        fig_flor, ax_flor = plt.subplots(figsize=(13,6))
        floresta_gleba.plot(ax=ax_flor, facecolor=cor_Floresta)
        gleba.plot(ax=ax_flor, facecolor='none', edgecolor='black', linewidth=1)
        # LEGENDAS
        patch_list = []
        patch_list.append(patches.Patch(facecolor='none', edgecolor='black', label='Gleba Federal', linewidth=1))
        patch_list.append(patches.Patch(facecolor=cor_Floresta, alpha=0.6, edgecolor='none', label='Floresta Pública', linewidth=1))
        # Creates a legend with the list of patches above.
        ax_flor.legend(handles=patch_list, fontsize=10, loc='lower left',
                bbox_to_anchor = (1,0.75),title='Legenda', title_fontsize=12)
        plt.xlabel('Longitude (°)')
        plt.ylabel('Latitude (°)')
        plt.grid()
        plt.show()
        floresta_gleba = floresta_gleba.rename(columns={'nome':'Nome','tipo':'Tipo','categoria':'Categoria'})
        display(floresta_gleba[['Nome', 'Tipo', 'Categoria']].to_html(index=False))
    else:
        display(Markdown('Não há sobreposição'))

    display(Markdown('### Unidades de Conservação'))
    uc_gleba = uc.sjoin(gleba, how='inner')
    if uc_gleba.shape[0]>0:
        fig_uc, ax_uc = plt.subplots(figsize=(13,6))
        uc_gleba.plot(ax=ax_uc, facecolor=cor_UC)
        gleba.plot(ax=ax_uc, facecolor='none', edgecolor='black', linewidth=1)
        # LEGENDAS
        patch_list = []
        patch_list.append(patches.Patch(facecolor='none', edgecolor='black', label='Gleba Federal', linewidth=1))
        patch_list.append(patches.Patch(facecolor=cor_UC, alpha=0.6, edgecolor='none', label='Unidade de Conservação', linewidth=1))
        # Creates a legend with the list of patches above.
        ax_uc.legend(handles=patch_list, fontsize=10, loc='lower left',
                bbox_to_anchor = (1,0.75),title='Legenda', title_fontsize=12)
        plt.xlabel('Longitude (°)')
        plt.ylabel('Latitude (°)')
        plt.grid()
        plt.show()
        uc_gleba = uc_gleba.rename(columns={'nome_uc1':'Nome','categori3':'Categoria','esfera5':'Responsabilidade'})
        display(uc_gleba[['Nome','Categoria','Responsabilidade']].to_html(index=False))
    else:
        display(Markdown('Não há sobreposição'))

    display(Markdown('### Terra Indígenas'))
    ti_gleba = ti.sjoin(gleba, how='inner')
    if ti_gleba.shape[0]>0:
        fig_ti, ax_ti = plt.subplots(figsize=(13,6))
        ti_gleba.plot(ax=ax_ti, facecolor=cor_TI)
        gleba.plot(ax=ax_ti, facecolor='none', edgecolor='black', linewidth=1)
        # LEGENDAS
        patch_list = []
        patch_list.append(patches.Patch(facecolor='none', edgecolor='black', label='Gleba Federal', linewidth=1))
        patch_list.append(patches.Patch(facecolor=cor_TI, alpha=0.6, edgecolor='none', label='Terra Indígena', linewidth=1))
        # Creates a legend with the list of patches above.
        ax_ti.legend(handles=patch_list, fontsize=10, loc='lower left',
                bbox_to_anchor = (1,0.75),title='Legenda', title_fontsize=12)
        plt.xlabel('Longitude (°)')
        plt.ylabel('Latitude (°)')
        plt.grid()
        plt.show()
        ti_gleba = ti_gleba.rename(columns={'terrai_nom':'Nome', 'fase_ti':'Fase de Regularização'})
        display(ti_gleba[['Nome', 'Fase de Regularização']].to_html(index=False))
    else:
        display(Markdown('Não há sobreposição'))

    display(Markdown('### Projetos de Assentamento'))
    pa_gleba = pa.sjoin(gleba, how='inner')
    if pa_gleba.shape[0] > 0:
        fig_pa, ax_pa = plt.subplots(figsize=(13,6))
        pa_gleba.plot(ax=ax_pa, facecolor=cor_PA)
        gleba.plot(ax=ax_pa, facecolor='none', edgecolor='black', linewidth=1)
        # LEGENDAS
        patch_list = []
        patch_list.append(patches.Patch(facecolor='none', edgecolor='black', label='Gleba Federal', linewidth=1))
        patch_list.append(patches.Patch(facecolor=cor_PA, alpha=0.6, edgecolor='none', label='Assentamento', linewidth=1))
        # Creates a legend with the list of patches above.
        ax_pa.legend(handles=patch_list, fontsize=10, loc='lower left',
                bbox_to_anchor = (1,0.75),title='Legenda', title_fontsize=12)
        plt.xlabel('Longitude (°)')
        plt.ylabel('Latitude (°)')
        plt.grid()
        plt.show()
        pa_gleba = pa_gleba.rename(columns={'cd_sipra':'SIPRA','nome_proje':'Nome','municipio':'Município'})
        display(pa_gleba[['SIPRA','Nome','Município']].to_html(index=False))
    else:
        display(Markdown('Não há sobreposição'))
    
    display(Markdown('### Território Quilombola'))
    tq_gleba = tq.sjoin(gleba, how='inner')
    if tq_gleba.shape[0] >0:
        fig_tq, ax_tq = plt.subplots(figsize=(13,6))
        tq_gleba.plot(ax=ax_tq, facecolor=cor_TQ)
        gleba.plot(ax=ax_tq, facecolor='none', edgecolor='black', linewidth=1)
                    # LEGENDAS
        patch_list = []
        patch_list.append(patches.Patch(facecolor='none', edgecolor='black', label='Gleba Federal', linewidth=1))
        patch_list.append(patches.Patch(facecolor=cor_TQ, alpha=0.6, edgecolor='none', label='Território Quilombola', linewidth=1))
        # Creates a legend with the list of patches above.
        ax_ti.legend(handles=patch_list, fontsize=10, loc='lower left',
                bbox_to_anchor = (1,0.75),title='Legenda', title_fontsize=12)
        plt.xlabel('Longitude (°)')
        plt.ylabel('Latitude (°)')
        plt.grid()
        plt.show()
        tq_gleba = tq_gleba.rename(columns={'nm_comunid':'Nome', 'esfera':'Responsabilidade', 'fase':'Fase da Regularização'})
        display(tq_gleba[['Nome', 'Responsabilidade', 'Fase da Regularização']].to_html(index=False))
    else:
        display(Markdown('Não há sobreposição'))
    
    display(Markdown('### SIGEF'))
    sigef_gleba = sigef.sjoin(gleba, how='inner')
    if sigef_gleba.shape[0] > 0:
        fig_sigef, ax_sigef = plt.subplots(figsize=(13,6))
        sigef_gleba.plot(ax=ax_sigef, facecolor=cor_SIGEF)
        gleba.plot(ax=ax_sigef, facecolor='none', edgecolor='black', linewidth=1)
                    # LEGENDAS
        patch_list = []
        patch_list.append(patches.Patch(facecolor='none', edgecolor='black', label='Gleba Federal', linewidth=1))
        patch_list.append(patches.Patch(facecolor=cor_SIGEF, alpha=0.6, edgecolor='none', label='SIGEF', linewidth=1))
        # Creates a legend with the list of patches above.
        ax_sigef.legend(handles=patch_list, fontsize=10, loc='lower left',
                bbox_to_anchor = (1,0.75),title='Legenda', title_fontsize=12)
        plt.xlabel('Longitude (°)')
        plt.ylabel('Latitude (°)')
        plt.grid()
        plt.show()
        sigef_gleba = sigef_gleba.rename(columns={'qrcode':'Código SIGEF','natureza':'Natureza do Polígono'})
        display(sigef_gleba[['Código SIGEF','Natureza do Polígono']].to_html(index=False))
    else:
        display(Markdown('Não há sobreposição'))

:::
<div>